In [15]:
import pandas as pd
from datetime import datetime
from datetime import timedelta     

In [16]:
NGR=pd.read_csv('playerstats.csv')

In [17]:
# Deletando a última linha do DataFrame
NGR = NGR.drop(NGR.index[-1])

In [18]:
NGR["First deposit"] =pd.to_datetime(NGR["First deposit"], errors = 'coerce')

In [19]:
NGR["Advertiser"].unique()

array(['3snet18432', 'defaultgalera.bet', 'mobytize', '3snet999',
       'AlfaleadsCPA', 'Affunlim', 'marketin', 'IdentiBRDisplay', 'bm22',
       'ingoogleads', 'ally0034', 'crmbonus', 'ally0004', 'ally0011',
       'ally0031', 'ally0032', 'ally0015', 'seoglr', 'gohike', 'liqesGA',
       'intaboola', 'romeos', 'vmmeightroom', 'rubju300', 'betandealpt',
       'knownmedia', 'supera088', 'fernando-mktesportivo', 'Gn8383',
       'LiqesGA_Mines', 'nsnwab', 'aff.account.latam.001', 'intimads',
       '3snet1010', 'bc22ppc2', 'PremierLeads2', 'faith', 'cleveradv',
       'julia.advertise.pro', 'leadbitolga', 'samir', 'liqes', 'ally0021',
       'allyinf0011', 'breakmedia', '365scores', 'jafe1000',
       'inpropellerads', 'EonF.Galera', 'jompa', 'PropellerAds2',
       'dmidd47', 'glaununes', 'msbtt', 'ally0037', 'marcus.igoalcorp',
       'ally0023', 'oddsscannerbr', 'g1odds', 'ally0012',
       'GaleraLimayPPC', 'premierleads', 'paymentsbuenpasomedia',
       'ally0038', 'ally00010', 'b

In [20]:
NGR["Calculated_NGR"] = NGR['Casino Real Bets'] - NGR['Casino Real Wins'] -NGR['Redeemed bonus initial amounts'] - NGR['Redeemed bonus pending winnings']

In [21]:
NGR['First deposit date abreviated'] =NGR["First deposit"].dt.strftime('%m/%Y')

In [22]:
NGR.columns.values

array(['Username', 'Country', 'Advertiser', 'Signed up', 'First deposit',
       'Last deposit date', 'Wants mail', 'Deposits', 'Deposit count',
       'Average deposit', 'Approved Withdrawals',
       'Approved Withdrawal count', 'Casino NGR', 'Casino bonuses',
       'Casino bonus count', 'Redeemed bonus initial amounts',
       'Redeemed bonus pending winnings', 'Poker bonus reverses',
       'Casino progressive share', 'Casino progressive wins',
       'Casino Bets', 'Casino Wins', 'Casino Real Bets',
       'Casino Real Wins', 'Casino game count', 'Casino bonuses removed',
       'Frozen', 'Sportsbook wins', 'Sportsbook bets',
       'Sportsbook gross income', 'Calculated_NGR',
       'First deposit date abreviated'], dtype=object)

In [23]:
output = NGR.groupby('First deposit date abreviated')[["Calculated_NGR","Deposits","Deposit count"]].sum().reset_index()

In [24]:
output.loc[len(output)] = ["Total",output["Calculated_NGR"].sum(),output["Deposits"].sum(),output["Deposit count"].sum()]



In [25]:
import requests
# Conversão para DataFrame
df = pd.DataFrame(output)

# Renomeando colunas
df.rename(columns={
    "First deposit date abreviated": "Mês",
    "Calculated_NGR": "NGR",
    "Deposits": "Depósitos"
}, inplace=True)

# Fazendo a requisição para a API de câmbio (exemplo com Exchange Rates API)
api_key = "YOUR_API_KEY"  # Substitua pela sua chave de API
url = f"https://api.exchangerate-api.com/v4/latest/BRL"
response = requests.get(url)
data = response.json()

# Obter a taxa de conversão de BRL para USD
exchange_rate = data["rates"]["USD"]

# Adicionando colunas em USD
df["NGR_USD"] = df["NGR"] * exchange_rate
df["Depósitos_USD"] = df["Depósitos"] * exchange_rate

# Formatando os valores em R$ e em US$
df["NGR"] = df["NGR"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df["Depósitos"] = df["Depósitos"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df["NGR_USD"] = df["NGR_USD"].apply(lambda x: f'${x:,.2f}')
df["Depósitos_USD"] = df["Depósitos_USD"].apply(lambda x: f'${x:,.2f}')

df

,Mês,NGR,Depósitos,Deposit count,NGR_USD,Depósitos_USD
0,01/2022,"R$134,69","R$533,44",14,$23.30,$92.29
1,01/2023,"R$11.135,44","R$21.945,81",360,"$1,926.43","$3,796.63"
2,01/2024,"R$-7.639,97","R$29.524,02",514,"$-1,321.71","$5,107.66"
3,02/2022,"R$-1.638,04","R$7.368,71",95,$-283.38,"$1,274.79"
4,02/2023,"R$5.319,54","R$19.976,39",319,$920.28,"$3,455.92"
5,02/2024,"R$3.600,69","R$29.606,30",519,$622.92,"$5,121.89"
6,03/2022,"R$-56,84","R$5.778,47",137,$-9.83,$999.68
7,03/2023,"R$3.630,63","R$13.143,71",255,$628.10,"$2,273.86"
8,03/2024,"R$15.345,15","R$31.784,28",461,"$2,654.71","$5,498.68"
9,04/2022,"R$18.205,07","R$40.242,66",294,"$3,149.48","$6,961.98"


In [26]:
import pandas as pd
import requests
# Supondo que df já está criado conforme o exemplo dado

# Conversão para DataFrame
df = pd.DataFrame(output)

# Renomeando colunas
df.rename(columns={
    "First deposit date abreviated": "Mês",
    "Calculated_NGR": "NGR",
    "Deposits": "Depósitos"
}, inplace=True)

# Separando o ano e mês
df['Ano'] = df['Mês'].str.split('/').str[1]
df['Mês_num'] = df['Mês'].str.split('/').str[0]


# Fazendo a requisição para a API de câmbio (exemplo com Exchange Rates API)
api_key = "YOUR_API_KEY"  # Substitua pela sua chave de API
url = f"https://api.exchangerate-api.com/v4/latest/BRL"
response = requests.get(url)
data = response.json()

# Obter a taxa de conversão de BRL para USD
exchange_rate = data["rates"]["USD"]

# Adicionando colunas em USD
df["NGR_USD"] = df["NGR"] * exchange_rate
df["Depósitos_USD"] = df["Depósitos"] * exchange_rate

# Filtrando dados por ano (2022, 2023 e ano atual 2024)
df_2022 = df[df['Ano'] == '2022'].copy()
df_2023 = df[df['Ano'] == '2023'].copy()
df_ano_atual = df[df['Ano'] == '2024']

# Somando os valores por ano, incluindo conversão para USD
sum_2022 = {
    'Mês': '2022',
    'NGR': f"R${df_2022['NGR'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Depósitos': f"R${df_2022['Depósitos'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'NGR_USD': f"${df_2022['NGR_USD'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Depósitos_USD': f"${df_2022['Depósitos_USD'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Deposit count': df_2022['Deposit count'].sum()
}

sum_2023 = {
    'Mês': '2023',
    'NGR': f"R${df_2023['NGR'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Depósitos': f"R${df_2023['Depósitos'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'NGR_USD': f"${df_2023['NGR_USD'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Depósitos_USD': f"${df_2023['Depósitos_USD'].sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Deposit count': df_2023['Deposit count'].sum()
}

# Calculando o total geral
total_geral = {
    'Mês': 'Total',
    'NGR': f"R${pd.concat([df_2022['NGR'], df_2023['NGR'], df_ano_atual['NGR']]).sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Depósitos': f"R${pd.concat([df_2022['Depósitos'], df_2023['Depósitos'], df_ano_atual['Depósitos']]).sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'NGR_USD': f"${pd.concat([df_2022['NGR_USD'], df_2023['NGR_USD'], df_ano_atual['NGR_USD']]).sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Depósitos_USD': f"${pd.concat([df_2022['Depósitos_USD'], df_2023['Depósitos_USD'], df_ano_atual['Depósitos_USD']]).sum():,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
    'Deposit count': df_2022['Deposit count'].sum() + df_2023['Deposit count'].sum() + df_ano_atual['Deposit count'].sum()
}

# Removendo o ano de cada mês do ano atual (mantendo o formato "MM/AAAA")
df_ano_atual['Mês'] = df_ano_atual['Mês_num'] + '/' + df_ano_atual['Ano']

# Concatenando os resultados e adicionando colunas de conversão para USD
df_final = pd.concat([pd.DataFrame([sum_2022]), pd.DataFrame([sum_2023]), df_ano_atual, pd.DataFrame([total_geral])], ignore_index=True)

# Adicionando a coluna de ordem
df_final['Ordem'] = range(1, len(df_final) + 1)

# Reordenando as colunas para colocar "Ordem" como a primeira coluna
df_final = df_final[['Ordem', 'Mês', 'NGR', 'Depósitos', 'NGR_USD', 'Depósitos_USD', 'Deposit count']]

# Exibindo o resultado final
df_final

C:\Users\Paulo\AppData\Local\Temp\ipykernel_8616\3534059534.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ano_atual['Mês'] = df_ano_atual['Mês_num'] + '/' + df_ano_atual['Ano']


,Ordem,Mês,NGR,Depósitos,NGR_USD,Depósitos_USD,Deposit count
0,1,2022,"R$79.909,32","R$301.652,65","$13.824,31","$52.185,91",4280
1,2,2023,"R$107.050,89","R$254.832,27","$18.519,80","$44.085,98",3941
2,3,01/2024,-7639.97,29524.02,-1321.71481,5107.65546,514
3,4,02/2024,3600.69,29606.3,622.91937,5121.8899,519
4,5,03/2024,15345.15,31784.28,2654.71095,5498.68044,461
5,6,04/2024,9208.12,34503.67,1593.00476,5969.13491,617
6,7,05/2024,19551.31,54596.26,3382.37663,9445.15298,978
7,8,06/2024,7655.71,68741.04,1324.43783,11892.19992,1173
8,9,07/2024,31449.21,67356.32,5440.71333,11652.64336,1149
9,10,08/2024,17427.8,92271.91,3015.0094,15963.04043,1427


In [27]:
# Aplicando a formatação de moeda para o ano atual
df_ano_atual["NGR"] = df_ano_atual["NGR"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df_ano_atual["Depósitos"] = df_ano_atual["Depósitos"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df_ano_atual["NGR_USD"] = df_ano_atual["NGR_USD"].apply(lambda x: f'${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df_ano_atual["Depósitos_USD"] = df_ano_atual["Depósitos_USD"].apply(lambda x: f'${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))

# Concatenando os resultados e adicionando colunas de conversão para USD
df_final = pd.concat([pd.DataFrame([sum_2022]), pd.DataFrame([sum_2023]), df_ano_atual, pd.DataFrame([total_geral])], ignore_index=True)

# Adicionando a coluna de ordem
df_final['Ordem'] = range(1, len(df_final) + 1)

# Reordenando as colunas para colocar "Ordem" como a primeira coluna
df_final = df_final[['Ordem', 'Mês', 'NGR', 'Depósitos', 'NGR_USD', 'Depósitos_USD', 'Deposit count']]

df_final['Data'] = "17/11/2024"


# Exibindo o resultado final
df_final


C:\Users\Paulo\AppData\Local\Temp\ipykernel_8616\2819486948.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ano_atual["NGR"] = df_ano_atual["NGR"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
C:\Users\Paulo\AppData\Local\Temp\ipykernel_8616\2819486948.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ano_atual["Depósitos"] = df_ano_atual["Depósitos"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
C:\Users\Paulo\AppData

,Ordem,Mês,NGR,Depósitos,NGR_USD,Depósitos_USD,Deposit count,Data
0,1,2022,"R$79.909,32","R$301.652,65","$13.824,31","$52.185,91",4280,17/11/2024
1,2,2023,"R$107.050,89","R$254.832,27","$18.519,80","$44.085,98",3941,17/11/2024
2,3,01/2024,"R$-7.639,97","R$29.524,02","$-1.321,71","$5.107,66",514,17/11/2024
3,4,02/2024,"R$3.600,69","R$29.606,30","$622,92","$5.121,89",519,17/11/2024
4,5,03/2024,"R$15.345,15","R$31.784,28","$2.654,71","$5.498,68",461,17/11/2024
5,6,04/2024,"R$9.208,12","R$34.503,67","$1.593,00","$5.969,13",617,17/11/2024
6,7,05/2024,"R$19.551,31","R$54.596,26","$3.382,38","$9.445,15",978,17/11/2024
7,8,06/2024,"R$7.655,71","R$68.741,04","$1.324,44","$11.892,20",1173,17/11/2024
8,9,07/2024,"R$31.449,21","R$67.356,32","$5.440,71","$11.652,64",1149,17/11/2024
9,10,08/2024,"R$17.427,80","R$92.271,91","$3.015,01","$15.963,04",1427,17/11/2024


In [28]:
import uuid

# Carregar o arquivo CSV existente
df_existing = pd.read_excel("daily_playerstats1.xlsx")

# Filtrar as colunas necessárias (incluindo 'ID')
df_existing = df_existing[["Mês", "NGR", "Depósitos", "Deposit count", "NGR_USD", "Depósitos_USD", "Data", "ID", "Ordem"]]

# Gerar um ID único para toda a nova tabela 'final_result'
unique_id = str(uuid.uuid4())

# Adicionar a mesma ID para todas as linhas da nova tabela
df_final['ID'] = unique_id

# Concatenar o DataFrame novo (final_result) ao existente
df_combined = pd.concat([df_existing, df_final], ignore_index=True)

# Remover duplicatas com base na coluna "Username" (ou outra coluna de sua escolha)
#df_combined = df_combined.drop_duplicates(subset=["Ordem"], keep="last")

# Salvar o DataFrame combinado e sem duplicatas no arquivo CSV
df_combined.to_excel("daily_playerstats1.xlsx",index=False)